# Hospital Lat Long Generator



In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmplot
import gmaps.datasets
import os
import json
import time



### Store results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "c://Users/shawn/Project 3//hospitalacquiredinfections2.csv"
HAI_data = pd.read_csv(file)
HAI_data.head()


,FacilityID,FacilityName,Address,City,State,CountyName,ZIPCode,PhoneNumber,% Catheter Associated Urinary Tract Infections,% Central Line Associated Bloodstream Infections,...,Clostridium Difficile (C.Diff) Numerator,Clostridium Difficile (C.Diff) Denominator,MRSA Bacteremia Numerator,MRSA Bacteremia Denominator,SSI - Abdominal Hysterectomy Numerator,SSI - Abdominal Hysterectomy Denominator,SSI - Colon Surgery Numerator,SSI - Colon Surgery Denominator,Sum of Total Numerator,Sum of Total Denominator
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,Houston,36301,(334) 793-8701,0.000402522,0.000579307,...,46,103069,7,103069,0,224,2,179,66,230078
1,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,Marshall,35957,(256) 593-8310,0.000581125,0.001047852,...,6,33275,4,35424,0,33,1,90,19,80289
2,10006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,Lauderdale,35630,(256) 768-8400,0.000478744,0.00095511,...,6,63359,6,64010,0,26,0,82,23,144203
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,Covington,36467,(334) 493-3541,0,0,...,0,5445,0,5445,0,0,0,8,0,12357
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,Crenshaw,36049,(334) 335-3374,0,0,...,0,3653,0,4337,0,0,0,0,0,8472


In [3]:
#Rename dataframe and add Lat & Long placeholder columns
hospital_geocoords = HAI_data
hospital_geocoords["Lat"] = ""
hospital_geocoords["Lng"] = ""
hospital_geocoords.head()

,FacilityID,FacilityName,Address,City,State,CountyName,ZIPCode,PhoneNumber,% Catheter Associated Urinary Tract Infections,% Central Line Associated Bloodstream Infections,...,MRSA Bacteremia Numerator,MRSA Bacteremia Denominator,SSI - Abdominal Hysterectomy Numerator,SSI - Abdominal Hysterectomy Denominator,SSI - Colon Surgery Numerator,SSI - Colon Surgery Denominator,Sum of Total Numerator,Sum of Total Denominator,Lat,Lng
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,Houston,36301,(334) 793-8701,0.000402522,0.000579307,...,7,103069,0,224,2,179,66,230078,,
1,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,Marshall,35957,(256) 593-8310,0.000581125,0.001047852,...,4,35424,0,33,1,90,19,80289,,
2,10006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,Lauderdale,35630,(256) 768-8400,0.000478744,0.00095511,...,6,64010,0,26,0,82,23,144203,,
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,Covington,36467,(334) 493-3541,0,0,...,0,5445,0,0,0,8,0,12357,,
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,Crenshaw,36049,(334) 335-3374,0,0,...,0,4337,0,0,0,0,0,8472,,


In [ ]:
# create a params dict that will be updated with address information each iteration
from api_keys import gkey
params = {"key": gkey}

# Loop through the hospital_geocoords dataframe and run a lat/long search for each city
for index, row in hospital_geocoords.iterrows():
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    Address = row['Address']
    City = row['City']
    State = row['State']
    Zip = row ['ZIPCode']

    # update address key value
    params['address'] = f"{Address}+{City}+{State}+{Zip}"

    # make request
    hospitals_lat_lng = requests.get(url, params=params)


    
    # convert to json
    hospitals_lat_lng = hospitals_lat_lng.json()
    
#Search for/select first result from API and store in 'Lat & Long' columns
#If KeyError or IndexError present, print "Missing field/result... skipping." and continue iteration

    try:
        hospital_geocoords.loc[index, "Lat"] = hospitals_lat_lng["results"][0]["geometry"]["location"]["lat"]
        hospital_geocoords.loc[index, "Lng"] = hospitals_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")  
   
    time.sleep(.5)


In [ ]:
hospital_geocoords.head()